Data downloaded from https://old.chesstempo.com/gamedb/player/67433.

In [1]:
import chess
import chess.pgn

import matplotlib, seaborn
from matplotlib import pyplot as plt

from scipy import stats

In [2]:
ref_player = 'Gajewski'
def count_pieces(board, game, player=ref_player, piece=chess.KNIGHT):
    # return number of given pieces for a given player on a given board position 
    if player == ref_player:
        if ref_player in game.headers['White']:
            return len(board.pieces(piece_type=piece, color=chess.WHITE))
        return len(board.pieces(piece_type=piece, color=chess.BLACK))
    else:
        if ref_player in game.headers['White']:
            return len(board.pieces(piece_type=piece, color=chess.BLACK))
        return len(board.pieces(piece_type=piece, color=chess.WHITE))

In [3]:
def grzesiu_won(game, draw_winning=False):
    player_white = game.headers['White']
    player_black = game.headers['Black']
    result = game.headers['Result']
    if ref_player in player_white:
        if not draw_winning: ## only full point wins
            if result == '1-0':
                return True
            return False
        else:
            if result in ('1-0', '1/2-1/2'):
                return True
            return False
    else:
        if not draw_winning:
            if result == '0-1':
                return True
            return False
        else:
            if result in ('0-1', '1/2-1/2'):
                return True
            return False

In [4]:
def piece_moved(board, last_move):
    pos = str(last_move)[2:4]
    piece = board.piece_at(chess.parse_square(pos))
    return str(piece)

In [117]:
def get_contingency_table(games):
    a = games['won']
    b = games['lost']
    c = games['won total'] - games['won']
    d = games['lost total'] - games['lost']
    return [[a,b], [c,d]]

In [158]:
pgn_file = 'data/gajewski_games.pgn'

In [159]:
# filtering out some games
years = []
output_pgn = 'data/gajewski_games_filtered.pgn'
with open(output_pgn, 'w') as f:
    with open(pgn_file) as pgn:
        while 1:
            game = chess.pgn.read_game(pgn)
            if not game:
                break
            year = int(game.headers['Date'].split('.')[0])
            years.append(year)
            if year >= 2015:
                continue
            print(game, file=f, end="\n\n")
pgn_file = output_pgn

In [160]:
minY = min(years)
maxY = max(years)
for Y in range(minY, maxY+1):
    print(Y, years.count(Y), sep='\t')

2006	1
2007	3
2008	10
2009	7
2010	12
2011	6
2012	24
2013	21
2014	28
2015	40
2016	24
2017	25
2018	11
2019	41
2020	11


In [161]:
features = ['Majority of knights in the end position',
            'Majority of bishops in the end position',
            'Majority of pawns in the end position',
            'Majority of knights after first 30 moves',
            'More than 10 knight moves made by Grzesiu',
            "Grzesiu's knight more active than opponent's",
            "Grzesiu's knight more active than opponent's in the last 10 moves"
           ]

tables = []

In [162]:
# Majority of knights in the end position

games = {'won total': 0, 'lost total': 0, 'won': 0, 'lost': 0}

with open(pgn_file) as pgn:
    while 1:
        game = chess.pgn.read_game(pgn)
        if not game:
            break

        board = game.board()
        for move in game.mainline_moves():
            board.push(move)

        # last state on the board
        g_knights = count_pieces(board, game, ref_player, chess.KNIGHT)
        o_knights = count_pieces(board, game, 'other', chess.KNIGHT)

        if grzesiu_won(game):
            if g_knights > o_knights:
                games['won'] += 1
            games['won total'] += 1
        else:
            if g_knights > o_knights:
                games['lost'] += 1
            games['lost total'] += 1

    ct = get_contingency_table(games)
    tables.append(ct)

In [163]:
# Majority of bishops in the end position

games = {'won total': 0, 'lost total': 0, 'won': 0, 'lost': 0}

with open(pgn_file) as pgn:
    while 1:
        game = chess.pgn.read_game(pgn)
        if not game:
            break

        board = game.board()
        for move in game.mainline_moves():
            board.push(move)

        # last state on the board
        g_bishops = count_pieces(board, game, ref_player, chess.BISHOP)
        o_bishops = count_pieces(board, game, 'other', chess.BISHOP)

        if grzesiu_won(game):
            if g_bishops > o_bishops:
                games['won'] += 1
            games['won total'] += 1
        else:
            if g_bishops > o_bishops:
                games['lost'] += 1
            games['lost total'] += 1

    ct = get_contingency_table(games)
    tables.append(ct)

In [164]:
# Majority of pawns in the end position

games = {'won total': 0, 'lost total': 0, 'won': 0, 'lost': 0}

with open(pgn_file) as pgn:
    while 1:
        game = chess.pgn.read_game(pgn)
        if not game:
            break

        board = game.board()
        for move in game.mainline_moves():
            board.push(move)

        # last state on the board
        g_pawns = count_pieces(board, game, ref_player, chess.PAWN)
        o_pawns = count_pieces(board, game, 'other', chess.PAWN)

        if grzesiu_won(game):
            if g_pawns > o_pawns:
                games['won'] += 1
            games['won total'] += 1
        else:
            if g_pawns > o_pawns:
                games['lost'] += 1
            games['lost total'] += 1

    ct = get_contingency_table(games)
    tables.append(ct)

In [165]:
# Majority of knights after first 30 moves

games = {'won total': 0, 'lost total': 0, 'won': 0, 'lost': 0}

with open(pgn_file) as pgn:
    while 1:
        game = chess.pgn.read_game(pgn)
        if not game:
            break

        board = game.board()
        for i,move in enumerate(game.mainline_moves()):
            if i > 29:
                break
            board.push(move)

        # last state on the board
        g_knights = count_pieces(board, game, ref_player, chess.KNIGHT)
        o_knights = count_pieces(board, game, 'other', chess.KNIGHT)

        if grzesiu_won(game):
            if g_knights > o_knights:
                games['won'] += 1
            games['won total'] += 1
        else:
            if g_knights > o_knights:
                games['lost'] += 1
            games['lost total'] += 1

    ct = get_contingency_table(games)
    tables.append(ct)

In [166]:
# More than 6 knight moves made by Grzesiu

games = {'won total': 0, 'lost total': 0, 'won': 0, 'lost': 0}

with open(pgn_file) as pgn:
    while 1:
        game = chess.pgn.read_game(pgn)
        if not game:
            break

        player_white = game.headers['White']
        player_black = game.headers['Black']

        g_knights, o_knights = 0, 0

        board = game.board()
        for i,move in enumerate(game.mainline_moves()):
            board.push(move)
            piece = piece_moved(board, move)
            if piece in 'Nn':
                if ref_player in player_white: # Grzesiu playing white
                    if piece == 'N':
                        g_knights += 1
                    else:
                        o_knights += 1
                else: # Grzesiu playing black
                    if piece == 'n':
                        g_knights += 1
                    else:
                        o_knights += 1

        if grzesiu_won(game):
            if g_knights > 6:
                games['won'] += 1
            games['won total'] += 1
        else:
            if g_knights > 6:
                games['lost'] += 1
            games['lost total'] += 1

    ct = get_contingency_table(games)
    tables.append(ct)

In [167]:
# Grzesiu's knight more active than opponent's

games = {'won total': 0, 'lost total': 0, 'won': 0, 'lost': 0}

with open(pgn_file) as pgn:
    while 1:
        game = chess.pgn.read_game(pgn)
        if not game:
            break

        player_white = game.headers['White']
        player_black = game.headers['Black']

        g_knights, o_knights = 0, 0

        board = game.board()
        for i,move in enumerate(game.mainline_moves()):
            board.push(move)
            piece = piece_moved(board, move)
            if piece in 'Nn':
                if ref_player in player_white: # Grzesiu playing white
                    if piece == 'N':
                        g_knights += 1
                    else:
                        o_knights += 1
                else: # Grzesiu playing black
                    if piece == 'n':
                        g_knights += 1
                    else:
                        o_knights += 1

        if grzesiu_won(game):
            if g_knights > o_knights:
                games['won'] += 1
            games['won total'] += 1
        else:
            if g_knights > o_knights:
                games['lost'] += 1
            games['lost total'] += 1

    ct = get_contingency_table(games)
    tables.append(ct)

In [168]:
# Grzesiu's knight more active than opponent's in the last 10 moves

games = {'won total': 0, 'lost total': 0, 'won': 0, 'lost': 0}

with open(pgn_file) as pgn:
    while 1:
        game = chess.pgn.read_game(pgn)
        if not game:
            break

        player_white = game.headers['White']
        player_black = game.headers['Black']

        g_knights, o_knights = 0, 0

        board = game.board()
        moves_total = 0
        for x in game.mainline_moves():
            moves_total += 1
        for i,move in enumerate(game.mainline_moves()):            
            board.push(move)
            if i < moves_total - 10:
                continue
            piece = piece_moved(board, move)
            if piece in 'Nn':
                if ref_player in player_white: # Grzesiu playing white
                    if piece == 'N':
                        g_knights += 1
                    else:
                        o_knights += 1
                else: # Grzesiu playing black
                    if piece == 'n':
                        g_knights += 1
                    else:
                        o_knights += 1

        if grzesiu_won(game):
            if g_knights > o_knights:
                games['won'] += 1
            games['won total'] += 1
        else:
            if g_knights > o_knights:
                games['lost'] += 1
            games['lost total'] += 1

    ct = get_contingency_table(games)
    tables.append(ct)

In [169]:
print(*features, sep='\n')
print()

for i,table in enumerate(tables):
    print(features[i])
    print(table)
    print(stats.fisher_exact(table))

print()
print(sum(table[0])+sum(table[1]))

Majority of knights in the end position
Majority of bishops in the end position
Majority of pawns in the end position
Majority of knights after first 30 moves
More than 10 knight moves made by Grzesiu
Grzesiu's knight more active than opponent's
Grzesiu's knight more active than opponent's in the last 10 moves

Majority of knights in the end position
[[7, 10], [22, 73]]
(2.3227272727272728, 0.13776286928174516)
Majority of bishops in the end position
[[7, 14], [22, 69]]
(1.5681818181818181, 0.41370051740245906)
Majority of pawns in the end position
[[16, 11], [13, 72]]
(8.055944055944057, 1.9169052713822133e-05)
Majority of knights after first 30 moves
[[4, 12], [25, 71]]
(0.9466666666666667, 1.0)
More than 10 knight moves made by Grzesiu
[[9, 31], [20, 52]]
(0.7548387096774194, 0.6544186049142888)
Grzesiu's knight more active than opponent's
[[9, 28], [20, 55]]
(0.8839285714285714, 1.0)
Grzesiu's knight more active than opponent's in the last 10 moves
[[5, 14], [24, 69]]
(1.0267857142

In [175]:
15/55

0.2727272727272727

In [176]:
33/77

0.42857142857142855